# Data Exploration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely

In [ ]:
# Proportion of trips on this date as compared to Avg number of trips on the same day of week in baseline period i.e 1st

In [ ]:
df = pd.read_csv("covid_impact_on_airport_traffic.csv")

In [ ]:
df

In [ ]:
countries = df.groupby(["Country"])["Country"].count().reset_index(name = "Count Country")

# Retrieve Long and Lat Coordiantes


In [ ]:
def getLat(point):
    _, lat = point[6:-1].split(" ")
    return float(lat)

In [ ]:
def getLong(point):
    long, _ = point[6:-1].split(" ")
    return float(long)

In [ ]:
df["lat"] = df["Centroid"].map(getLat)
df["long"] = df["Centroid"].map(getLong)

In [ ]:
df

In [ ]:
# split dataframe into USA and CA
canada = df.loc[df["Country"] == "Canada"]
usa = df.loc[df["Country"] == "United States of America (the)"]
temp = [usa, canada]
na = pd.concat(temp)

In [ ]:
# cities in CA and USA
canadaCities = canada.groupby(["City"])["City"].count().reset_index(name = "Count City")
usaCities = usa.groupby(["City"])["City"].count().reset_index(name = "Count City")
naCities = na.groupby(["City"])["City"].count().reset_index(name = "Count City")

In [ ]:
na

In [ ]:
rankNA = na.groupby("AirportName").mean().sort_values(by = "PercentOfBaseline")
rankCA = canada.groupby("AirportName").mean().sort_values(by = "PercentOfBaseline")
rankUSA = usa.groupby("AirportName").mean().sort_values(by = "PercentOfBaseline")

In [ ]:
rankCA

# Plot Coordinates of Airports in North America

In [ ]:
# Refer to geopandas documentation: https://geopandas.org/gallery/create_geopandas_from_pandas.html
world = gpd.read_file(
    gpd.datasets.get_path('naturalearth_lowres')
)

In [ ]:
# Make 'Geometry' in NA dataframe; will have to refactor this later on (code is messy, have three cols of the same thing)
gdfna = gpd.GeoDataFrame(na, geometry = gpd.points_from_xy(na.long, na.lat))

In [ ]:
# Plot coordinates onto map of NA
ax = world[world.continent == 'North America'].plot(color='white', edgecolor='black')
gdfna.plot(ax = ax, color = 'blue', markersize = 10)
plt.savefig('na.svg')
#plt.show()

# Find Outliers 

In [ ]:
canada.boxplot(column = ['PercentOfBaseline'], by = 'AirportName', rot = 90, fontsize = 10, figsize = (18,10))

In [ ]:
usa.boxplot(column = ['PercentOfBaseline'], by = 'AirportName', rot = 90, fontsize = 10, figsize = (25,15))

In [ ]:
# Find monthly outliers of YVR 
canada['Date'] = pd.to_datetime(canada['Date'])
yvr = canada.loc[canada['AirportName'] == 'Vancouver International']

In [ ]:
# https://stackoverflow.com/questions/44908383/how-can-i-group-by-month-from-a-date-field-using-python-pandas/44908576
yvr.groupby(yvr['Date'].dt.strftime('%b'))['PercentOfBaseline'].mean().sort_values

In [ ]:
yvr.boxplot(column = ['PercentOfBaseline'], by = yvr['Date'].dt.strftime('%b'), rot = 90, fontsize = 10, figsize = (25,15))
plt.suptitle("")
plt.xlabel("")
# Repeat this for all NA airports?